In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from tqdm import tqdm

In [3]:
EXP_NAME = "exp153"
DIR = "/gdrive/MyDrive/amazon_kdd_2023/"
WEIGHTS = [1.0, 1.0, 1.0, 1.0, 1.0]

In [4]:
subs = {
    "exp127": DIR + "data/output/task2/exp127.parquet",
    "exp129": DIR + "data/output/task2/exp129.parquet",
    "exp132": DIR + "data/output/task2/exp132.parquet",
    "exp147": DIR + "data/output/task2/exp147.parquet",
    "exp151": DIR + "data/output/task2/exp151.parquet",
}

In [5]:
assert(len(WEIGHTS) == len(subs))

In [6]:
for i, n in enumerate(subs):
    path = subs[n]
    if i == 0:
        sub = pd.read_parquet(path)
        sub.columns = ['locale', 'labels0']
    else:
        sub[f'labels{i}'] = pd.read_parquet(path)['next_item_prediction']

In [7]:
def cust_blend(dt, W = [1,1,1,1], base=3):   
    REC = []
    for i in range(len(W)):
        REC.append(dt[f'labels{i}'])

    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+base))
            else:
                res[v] = (W[M]/(n+base))
    
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())
    
    return res[:100]

In [8]:
sub['next_item_prediction'] = sub.apply(cust_blend, W = WEIGHTS, axis=1)

In [9]:
sub[["locale", "next_item_prediction"]].to_parquet(DIR + "data/output/task2/" + EXP_NAME + ".parquet", engine='pyarrow')

In [10]:
sub.head()

,locale,labels0,labels1,labels2,labels3,labels4,next_item_prediction
0,ES,"[B074RZKWMD, B07457KX2S, B074581R8Y, B0745795K...","[B074RZKWMD, B0745795KF, B07457KX2S, B074581R8...","[B0745795KF, B074RZKWMD, B074581R8Y, B07457KX2...","[B074RZKWMD, B0745795KF, B07457KX2S, B074581R8...","[B0745795KF, B074RZKWMD, B074581R8Y, B07457KX2...","[B074RZKWMD, B0745795KF, B07457KX2S, B074581R8..."
1,ES,"[B09M8LNB61, B09J4T4JF5, B09NTR22SW, B09NQKCQW...","[B09M8LNB61, B09J4T4JF5, B09NTR22SW, B09NQKCQW...","[B09M8LNB61, B09J4T4JF5, B09NQKCQWS, B09NT33LZ...","[B09J4T4JF5, B09M8LNB61, B09NQKCQWS, B09NT33LZ...","[B09M8LNB61, B09J4T4JF5, B0998ZJCKF, B09NQKCQW...","[B09M8LNB61, B09J4T4JF5, B09NQKCQWS, B09NTR22S..."
2,ES,"[B0B461V1RB, B0B4614XL6, B0B4643ZJN, B0B4625TR...","[B0B461V1RB, B0B4614XL6, B0B4625TR5, B0B461KYH...","[B0B461V1RB, B0B4614XL6, B0B4643ZJN, B0B4625TR...","[B0B461V1RB, B0B4614XL6, B0B4643ZJN, B0B461KYQ...","[B0B461V1RB, B0B4614XL6, B0B4643ZJN, B0B4625TR...","[B0B461V1RB, B0B4614XL6, B0B4643ZJN, B0B4625TR..."
3,ES,"[B07R4VG9X3, B07R4WKSSV, B07QZ35DY6, B07QXZJK8...","[B07R4VG9X3, B07R4WKSSV, B07QZ35DY6, B07QXZJK8...","[B07R4VG9X3, B07R4WKSSV, B07QZ35DY6, B07QXZJK8...","[B07R4VG9X3, B07R4WKSSV, B07QZ35DY6, B07QXZJK8...","[B07R4WKSSV, B07R4VG9X3, B07QZ35DY6, B07QXZJK8...","[B07R4VG9X3, B07R4WKSSV, B07QZ35DY6, B07QXZJK8..."
4,ES,"[B09G9LF91K, B09G97SLGS, B09G99D95Q, B09G9DMQ7...","[B09G9LF91K, B09G97SLGS, B09G9DMQ7M, B09G99D95...","[B09G9LF91K, B09G97SLGS, B09G99D95Q, B09G9DMQ7...","[B09G9LF91K, B09G97SLGS, B09G9DMQ7M, B09G99D95...","[B09G9LF91K, B09G97SLGS, B09G9DMQ7M, B09G99D95...","[B09G9LF91K, B09G97SLGS, B09G9DMQ7M, B09G99D95..."
